In [1]:
import pandas as pd

import sys

sys.path.append("../")
import spell as s

sys.path.append('../../new_src/Thesis/refactored')
import Reader
import LogKeysManager
import ParamsExtractor2
import DataPreprocessor
import ReduceDim
import ClusterData

2024-06-15 09:02:44.255041: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-15 09:02:44.255686: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-15 09:02:44.260242: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-15 09:02:44.304710: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-15 09:02:45.105764: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF

In [2]:
import os
os.getcwd()

'/home/rocky/spell/pyspell'

In [3]:
#os.listdir('../../../../temp_logs/20240420')

In [4]:
# Testing Params Extractor
#log_types = ['cron', 'user']
log_types = ['cron', 'maillog', 'messages', 'secure', 'user']
file_paths = ['../../../../temp_logs/20240420/login01.' + logtype + '.log' for logtype in log_types]

df_list = []

for file_path in file_paths:
    r = Reader.Reader(file_path)
    df = r.read_file(file_path)
    df_list.append(df)

In [5]:
df_after = []

for i, df in enumerate(df_list):
    p = ParamsExtractor2.ParamsExtractor(df)
    df = p.convert_params(df)
    new_df = p.get_params()
    df_after.append(new_df)


for i in range(len(df_list)):
    df_list[i] = pd.concat([df_list[i], df_after[i]], axis=1)

In [6]:
df_list[0].columns

Index(['host', 'ident', 'pid', 'message', 'severity', 'facility', 'time',
       'severity_numbers', 'facility_numbers', 'severity_scores', 'timedelta',
       'log key', 'message'],
      dtype='object')

In [7]:
logs = [[m for m in df_after[i]['message']] for i in range(len(df_list))]
flattened_logs = [message for sublist in logs for message in sublist]

In [8]:
slm = s.lcsmap('[\\s]+')

'''
# Open the file 'test.log' and read lines from it
file_path = '../../../../temp_logs/'
with open('test/test.log', 'r') as file:
    lines = file.readlines()
'''

for i in flattened_logs:
    sub = i.strip('\n')
    obj = slm.insert(sub)
    #print(obj.get_id(), obj.param(sub))

s.save('slm.pickle', slm)

slm.dump()


0 {"lcsseq": "* * ", "lineids": [1, 4], "postion": [0, 1]}
1 {"lcsseq": "* * 2>&1 ", "lineids": [2, 3], "postion": [0, 1]}
2 {"lcsseq": "(*) * 0anacron ", "lineids": [5, 8, 22, 25, 40, 43, 59, 63, 84, 85, 102, 103, 122, 123, 143, 144, 164, 165, 186, 187, 207, 208, 231, 232, 259, 260, 286, 287, 312, 313, 338, 339, 364, 365, 391, 392, 417, 418, 438, 439, 459, 460, 481, 482, 509, 510, 535, 536], "postion": [1]}
3 {"lcsseq": "* * ", "lineids": [6, 21], "postion": [0, 1]}
4 {"lcsseq": "Normal exit (*) ", "lineids": [7, 24, 42, 79], "postion": []}
5 {"lcsseq": "* \"STATUS_CHECK\" -p local2.info >> * ", "lineids": [9, 15, 26, 31, 44, 51, 64, 71, 86, 94, 104, 111, 124, 133, 145, 154, 166, 176, 188, 197, 209, 218, 233, 246, 261, 273, 288, 299, 314, 325, 340, 351, 366, 378, 393, 404, 419, 428, 440, 449, 461, 471, 483, 494, 511, 521, 537, 547], "postion": [0, 5]}
6 {"lcsseq": "* --delay 20 --cronid * 1>* 2>&1 ", "lineids": [10, 11, 13, 14, 16, 18, 19, 27, 29, 30, 32, 35, 36, 45, 48, 49, 52, 55, 5

matching a new line

In [9]:
flattened_logs[2221]

'pam_unix(*): check pass; user *'

In [10]:
sub = flattened_logs[2221].strip('\n') 
obj = slm.match(sub)
print(obj.get_id(), obj.param(sub))

59 [['*']]
